In [3]:
#Importación de paqueterias

import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk import ngrams
import spacy
#nltk.download('wordnet')
#from nltk.stem import WordNetLemmatizer
#from sklearn.model_selection import train_test_split


In [9]:
#df_glass_completo=pd.read_csv("glassdoor_reviews.csv")

In [10]:
#Se hace un DF mas pequeño para trabajar mas facilemte
#df_glass_mil =df_glass_completo.head(1000)

#df_glass_mil.to_csv('glassdor_mil.csv', index=False)


In [11]:
#Crea df con prueba y test
#train_set = df_glass_mil.sample(frac=0.8, random_state=42) 
#train_set.to_csv('glassdor_train.csv', index=False)

# Drop de los datos que estan en el set train
 
#test_set = df_glass_mil.drop(train_set.index) 
#test_set.to_csv('glassdor_test.csv', index=False)


In [ ]:
#Se trabaja con el train df

df_glass=pd.read_csv("glassdor_train.csv")
df_glass.head(3)

In [13]:
#Replace NaN with , Na values interfere in the merge and re code

df_glass['headline'] = df_glass['headline'].fillna(",")
df_glass['pros'] = df_glass['pros'].fillna(",")
df_glass['cons'] = df_glass['cons'].fillna(",")

In [ ]:
#Merge hedline, pros, cons

df_glass["headline+pros+cons"] = df_glass["headline"] + " " + df_glass["pros"]+ " " + df_glass["cons"]

df_glass["headline+pros+cons"]

In [ ]:
#Check if there is an Na in "headline+pros+cons"

df_glass[df_glass["headline+pros+cons"].isnull()]

In [16]:
#Make the info at column "headline+pros+cons" string

df_glass['headline+pros+cons']= df_glass['headline+pros+cons'].astype('string')

# Text Preprocessing

In [17]:
#Remove Punctuation Marks

def quitar_puntuacion(texto):
    return re.sub(r'[^\w\s]',"", texto)

# Apply function to the merged column

df_glass['headline+pros+cons'] = df_glass['headline+pros+cons'].apply(quitar_puntuacion)

#Remove Stop words
def quitar_stop(texto):
    return re.sub(r'\b(?:and|an|at|a|of|on|I|for|with|the|at|from|in|to)\b', "", texto, flags=re.IGNORECASE)

# Apply function to the merged column

df_glass['headline+pros+cons'] = df_glass['headline+pros+cons'].apply(quitar_stop)



In [ ]:
#The info ended as type object, have to convert it to string
df_glass['headline+pros+cons']

In [19]:
#Lematize
nlp = spacy.load("en_core_web_sm")

def lematizar_(text):
    obj = nlp(text.lower())
    lemmatize_token = [x.lemma_ for x in obj]
    return lemmatize_token

# Aplicamos la lematización a cada fila
df_glass['headline+pros+cons_lemma'] = df_glass['headline+pros+cons'].apply(lematizar_)

In [20]:
#Hacer lemma string
df_glass['headline+pros+cons_lemma']=df_glass['headline+pros+cons_lemma'].astype('string')

In [21]:
#Remove Symbols Stop words
def quitar_stop(texto):
    return re.sub(r"[ \[\],']", " ", texto, flags=re.IGNORECASE)

# Apply function to the merged column

df_glass['headline+pros+cons_cleanlemma'] = df_glass['headline+pros+cons_lemma'].apply(quitar_stop)

In [22]:
#Create new DF with bigrams to record Y values (recommend)

df_bigrams_recommend=pd.DataFrame({"count":[],"Recommend":[]})

cc = 0

for x in df_glass['headline+pros+cons_cleanlemma']:

    text = df_glass['headline+pros+cons_cleanlemma'].iloc[cc]
    #recommend = df_glass['recommend'].iloc[cc]

    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))

    Ngrams=-1 #To record all

    #unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]

    bigram_counts = (pd.Series(bigrams).value_counts())[:Ngrams]
    
    #trigram_counts = (pd.Series(trigrams).value_counts())[:Ngrams]
    

    df_new = bigram_counts.to_frame()

    df_new["Recommend"]= df_glass['recommend'].iloc[cc]

    df_bigrams_recommend = pd.concat([df_new, df_bigrams_recommend])
    cc = cc+ 1


In [ ]:
bigram_counts

In [24]:
#Index a columna
df_bigrams_recommend=df_bigrams_recommend.reset_index()

In [25]:
#Save the DF with bigrams on a CSV
df_bigrams_recommend.to_csv('Bigrams.csv', index=False)

In [26]:
#Se quitan los duplicados del reporte de recommend, y se remueve la columna count

valores_bigrams_unicos = df_bigrams_recommend.drop_duplicates(subset=['index','Recommend'])
valores_bigrams_unicos = valores_bigrams_unicos.drop('count',axis=1)


In [ ]:
valores_bigrams_unicos

In [28]:
#Convertimos los valores de la tupla a string, y depues concatenamos

valores_bigrams_unicos["index"]=valores_bigrams_unicos["index"].astype(str)
valores_bigrams_unicos["index+Recommend"]=valores_bigrams_unicos["index"]+ valores_bigrams_unicos["Recommend"]

In [ ]:
valores_bigrams_unicos

In [30]:
#Agarramos el df de las recomendaciones y hacemos el concatenado anterior

df_bigrams_recommend_modificado=df_bigrams_recommend

df_bigrams_recommend_modificado["index"]=df_bigrams_recommend_modificado["index"].astype(str)
df_bigrams_recommend_modificado["index+Recommend"]=df_bigrams_recommend_modificado["index"]+ df_bigrams_recommend_modificado["Recommend"]

In [ ]:
df_bigrams_recommend_modificado

In [ ]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Bigrama + Reseña"

valores_bigrams_unicos["Frecuencia Bigrama + Reseña"]=0
loc=0

for gramm in valores_bigrams_unicos['index+Recommend']:
    cc=0
    loc2=0
    
    for gramm_2 in df_bigrams_recommend_modificado ['index+Recommend']:
        if gramm == gramm_2:
            cc= cc+ df_bigrams_recommend_modificado['count'].iloc[loc2]
        loc2=loc2+1
    
    valores_bigrams_unicos["Frecuencia Bigrama + Reseña"].iloc[loc]=cc
    loc=loc+1

In [ ]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Bigrama"

valores_bigrams_unicos["Frecuencia Bigrama"]=0
loc=0

for gramm in valores_bigrams_unicos['index']:
    cc=0
    loc2=0
    
    for gramm_2 in df_bigrams_recommend_modificado ['index']:
        if gramm == gramm_2:
            cc= cc+ df_bigrams_recommend_modificado['count'].iloc[loc2]
        loc2=loc2+1
    
    valores_bigrams_unicos["Frecuencia Bigrama"].iloc[loc]=cc
    loc=loc+1


In [ ]:
#Hacemos un ciclo For para calcular probabilidad

valores_bigrams_unicos["Probabilidad"]=0
loc=0

for fecuencia in valores_bigrams_unicos['index']:

    valores_bigrams_unicos["Probabilidad"].iloc[loc]=valores_bigrams_unicos["Frecuencia Bigrama + Reseña"].iloc[loc]/valores_bigrams_unicos["Frecuencia Bigrama"].iloc[loc]
    loc=loc+1

In [35]:
valores_bigrams_unicos

valores_bigrams_unicos.to_csv('ProbabilidadBiGramm.csv', index=False)

Hacer lo mismo de Bigrams a Unigrams

In [36]:
#Create new DF with unigrams to record Y values (recommend)

df_uigrams_recommend=pd.DataFrame({"count":[],"Recommend":[]})

cc = 0

for x in df_glass['headline+pros+cons_cleanlemma']:

    text = df_glass['headline+pros+cons_cleanlemma'].iloc[cc]
    #recommend = df_glass['recommend'].iloc[cc]

    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))
    #bigrams = list(ngrams(tokens, 2))
    #trigrams = list(ngrams(tokens, 3))

    Ngrams=-1 #To record all

    unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]

    #bigram_counts = (pd.Series(bigrams).value_counts())[:Ngrams]
    
    #trigram_counts = (pd.Series(trigrams).value_counts())[:Ngrams]
    

    df_new = unigram_counts.to_frame()

    df_new["Recommend"]= df_glass['recommend'].iloc[cc]

    df_uigrams_recommend = pd.concat([df_new, df_uigrams_recommend])
    cc = cc+ 1

In [ ]:
df_uigrams_recommend

In [38]:
#Index a columna
df_uigrams_recommend = df_uigrams_recommend.reset_index()

In [39]:
#Save the DF with unigrams on a CSV
df_uigrams_recommend.to_csv('Unigrams.csv', index=False)

In [40]:
#Se quitan los duplicados del reporte de recommend, y se remueve la columna count

valores_unigrams_unicos = df_uigrams_recommend.drop_duplicates(subset=['index','Recommend'])
valores_unigrams_unicos = valores_unigrams_unicos.drop('count',axis=1)

In [ ]:
valores_unigrams_unicos

In [42]:
#Convertimos los valores de la tupla a string, y depues concatenamos

valores_unigrams_unicos["index"]=valores_unigrams_unicos["index"].astype(str)
valores_unigrams_unicos["index+Recommend"]=valores_unigrams_unicos["index"]+ valores_unigrams_unicos["Recommend"]

In [43]:
#Agarramos el df de las recomendaciones y hacemos el concatenado anterior

df_uigrams_recommend_modificado=df_uigrams_recommend

df_uigrams_recommend_modificado["index"]=df_uigrams_recommend_modificado["index"].astype(str)
df_uigrams_recommend_modificado["index+Recommend"]=df_uigrams_recommend_modificado["index"]+ df_uigrams_recommend_modificado["Recommend"]

In [ ]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Unigrama + Reseña"

valores_unigrams_unicos["Frecuencia Unigrama + Reseña"]=0
loc=0

for gramm in valores_unigrams_unicos['index+Recommend']:
    cc=0
    loc2=0
    
    for gramm_2 in df_uigrams_recommend_modificado ['index+Recommend']:
        if gramm == gramm_2:
            cc= cc+ df_uigrams_recommend_modificado['count'].iloc[loc2]
        loc2=loc2+1
    
    valores_unigrams_unicos["Frecuencia Unigrama + Reseña"].iloc[loc]=cc
    loc=loc+1

In [ ]:
valores_unigrams_unicos

In [ ]:
#Hacemos un ciclo For para hacer la suma acumulada "Frecuencia Bigrama"

valores_unigrams_unicos["Frecuencia Unigrama"]=0
loc=0

for gramm in valores_unigrams_unicos['index']:
    cc=0
    loc2=0
    
    for gramm_2 in df_uigrams_recommend_modificado ['index']:
        if gramm == gramm_2:
            cc= cc+ df_uigrams_recommend_modificado['count'].iloc[loc2]
        loc2=loc2+1
    
    valores_unigrams_unicos["Frecuencia Unigrama"].iloc[loc]=cc
    loc=loc+1

In [ ]:
#Hacemos un ciclo For para calcular probabilidad

valores_unigrams_unicos["Probabilidad"]=0
loc=0

for fecuencia in valores_unigrams_unicos['index']:

    valores_unigrams_unicos["Probabilidad"].iloc[loc]=valores_unigrams_unicos["Frecuencia Unigrama + Reseña"].iloc[loc]/valores_unigrams_unicos["Frecuencia Unigrama"].iloc[loc]
    loc=loc+1

In [48]:
valores_unigrams_unicos

valores_unigrams_unicos.to_csv('ProbabilidadUniGramm.csv', index=False)


In [ ]:
valores_unigrams_unicos

In [50]:
#Hacer tabla prob limpia
df_uigrams_test=pd.DataFrame({"Word":[],"V":[],"X":[],"O":[]})
df_uigrams_test['Word']=valores_unigrams_unicos['index'].drop_duplicates()

In [ ]:
df_uigrams_test = df_uigrams_test.fillna(0)

df_uigrams_test

In [ ]:
locw=0
for word in df_uigrams_test['Word']:
    loci=0
    for index in valores_unigrams_unicos['index']:
        if word == index:
            if valores_unigrams_unicos['Recommend'].iloc[loci]=='v':
                df_uigrams_test['V'].iloc[locw]=valores_unigrams_unicos['Probabilidad'].iloc[loci]
            if valores_unigrams_unicos['Recommend'].iloc[loci]=='x':
                df_uigrams_test['X'].iloc[locw]=valores_unigrams_unicos['Probabilidad'].iloc[loci]
            if valores_unigrams_unicos['Recommend'].iloc[loci]=='o':
                df_uigrams_test['O'].iloc[locw]=valores_unigrams_unicos['Probabilidad'].iloc[loci]
        loci=loci+1
    locw=locw+1 
    

In [53]:
df_uigrams_test.to_csv('ProbabilidadUniGramm_VXO.csv', index=False)


## Modelo

In [ ]:
#Calcular probabilidades

df_puni=pd.read_csv("glassdor_train.csv")

vv=0
xx=0
oo=0

for reco in df_puni["recommend"]:
    if reco == "v":
        vv=vv+1
    if reco == "x":
        xx=xx+1   
    if reco == "o":
        oo=oo+1

tot_rec = vv+xx+oo
prob_v= vv/tot_rec
prob_x= xx/tot_rec
prob_o= oo/tot_rec

print(f'Probabilidad de ocurrencia V: {prob_v}')
print(f'Probabilidad de ocurrencia X: {prob_x}')
print(f'Probabilidad de ocurrencia O: {prob_o}')


In [55]:
#Se abre el df test

df_glass_t_test=pd.read_csv("glassdor_test.csv")
df_glass_t_test.head(5)

#-------Se le da formato--------------


df_glass_t_test['headline'] = df_glass_t_test['headline'].fillna(",")
df_glass_t_test['pros'] = df_glass_t_test['pros'].fillna(",")
df_glass_t_test['cons'] = df_glass_t_test['cons'].fillna(",")

#Merge hedline, pros, cons
df_glass_t_test["headline+pros+cons"] = df_glass_t_test["headline"] + " " + df_glass_t_test["pros"]+ " " + df_glass_t_test["cons"]
df_glass_t_test["headline+pros+cons"]

#Make the info at column "headline+pros+cons" string
df_glass_t_test['headline+pros+cons']= df_glass_t_test['headline+pros+cons'].astype('string')

#Remove Punctuation Marks
# Apply function to the merged column
df_glass_t_test['headline+pros+cons'] = df_glass_t_test['headline+pros+cons'].apply(quitar_puntuacion)

#Remove Stop words
# Apply function to the merged column
df_glass_t_test['headline+pros+cons'] = df_glass_t_test['headline+pros+cons'].apply(quitar_stop)

#Lematize
# Aplicamos la lematización a cada fila
nlp = spacy.load("en_core_web_sm")
df_glass_t_test['headline+pros+cons_lemma'] = df_glass_t_test['headline+pros+cons'].apply(lematizar_)

#Hacer lemma string
df_glass_t_test['headline+pros+cons_lemma']=df_glass_t_test['headline+pros+cons_lemma'].astype('string')

#Remove Symbols Stop words
# Apply function to the merged column
df_glass_t_test['headline+pros+cons_cleanlemma'] = df_glass_t_test['headline+pros+cons_lemma'].apply(quitar_stop)

In [56]:
#Crea una columna para ubicar a usuario
df_glass_t_test.index.name = 'User'
#Index a columna
df_glass_t_test = df_glass_t_test.reset_index()


In [ ]:
df_glass_t_test.head(5)

In [58]:
#Create new DF with unigrams to record Y values (recommend)

df_uigrams_test=pd.DataFrame({"count":[],"Recommend":[],"User":[]})

cc = 0

for x in df_glass_t_test['headline+pros+cons_cleanlemma']:

    text = df_glass_t_test['headline+pros+cons_cleanlemma'].iloc[cc]
        
    tokens = nltk.word_tokenize(text)

    unigrams=list(ngrams(tokens, 1))

    Ngrams=-1 #To record all

    unigram_counts = (pd.Series(unigrams).value_counts())[:Ngrams]
    

    df_new = unigram_counts.to_frame()

    df_new["Recommend"]= df_glass_t_test['recommend'].iloc[cc]
    df_new["User"]= df_glass_t_test['User'].iloc[cc]

    df_uigrams_test = pd.concat([df_new, df_uigrams_test])
    cc = cc+ 1

In [59]:
#Index a columna
df_uigrams_test = df_uigrams_test.reset_index()
#Save the DF with unigrams test on a CSV
df_uigrams_test.to_csv('Unigrams_test.csv', index=False)

In [ ]:
df_uigrams_test["index"]=df_uigrams_test["index"].astype(str)
df_uigrams_test

In [ ]:
#Abre csv Probabilidades
df_prob=pd.read_csv('ProbabilidadUniGramm_VXO.csv')
df_prob.head(2)


In [62]:
#Calcular Probabilidades Unigram

#Primero sacamos los usuarios unicos

usuarios_unicos=pd.DataFrame({"User":[],"ProbV":[],"ProbX":[],"ProbO":[]})

usuarios_unicos['User']=df_uigrams_test["User"].drop_duplicates()


In [ ]:
#Quita Na

usuarios_unicos = usuarios_unicos.fillna(0)

usuarios_unicos

In [ ]:
loccu=0
for user in usuarios_unicos['User']:
    locct=0
    for palabra_test in df_uigrams_test["index"]:
        if user == df_uigrams_test["User"].iloc[locct]:
            loccp=0
            for pal_prob in df_prob ["Word"]:
                if pal_prob == palabra_test:
                    if df_prob ["V"].iloc[loccp] != 0:
                        if usuarios_unicos['ProbV'].iloc[loccu]==0:
                            usuarios_unicos['ProbV'].iloc[loccu]=1
                        usuarios_unicos['ProbV'].iloc[loccu]= usuarios_unicos['ProbV'].iloc[loccu]*(df_prob ["V"].iloc[loccp]**df_uigrams_test["count"].iloc[locct])
                    
                    if df_prob ["X"].iloc[loccp] != 0:
                        if usuarios_unicos['ProbX'].iloc[loccu]==0:
                            usuarios_unicos['ProbX'].iloc[loccu]=1
                        usuarios_unicos['ProbX'].iloc[loccu]= usuarios_unicos['ProbX'].iloc[loccu]*(df_prob ["X"].iloc[loccp]**df_uigrams_test["count"].iloc[locct])
                    
                    if df_prob ["O"].iloc[loccp] != 0:
                        if usuarios_unicos['ProbO'].iloc[loccu]==0:
                            usuarios_unicos['ProbO'].iloc[loccu]=1
                        usuarios_unicos['ProbO'].iloc[loccu]= usuarios_unicos['ProbO'].iloc[loccu]*(df_prob ["O"].iloc[loccp]**df_uigrams_test["count"].iloc[locct])
                loccp=loccp+1
        locct=locct+1
    loccu=loccu+1



In [ ]:
print("Cálculo de numeradores por Usuario:")
usuarios_unicos

In [ ]:
#Normalizacion para encontrar las probabilidades reales

#Se saca el total de las probabilidades por usuario

usuarios_unicos["SumProb"]=0

locs=0

for usuario in usuarios_unicos["SumProb"]:
    usuarios_unicos["SumProb"].iloc[locs]=usuarios_unicos["ProbV"].iloc[locs]+usuarios_unicos["ProbX"].iloc[locs]+usuarios_unicos["ProbO"].iloc[locs]
    locs=locs+1
#Cálculo probabilidad normalizada
usuarios_unicos["ProbNomV%"]=0
usuarios_unicos["ProbNomX%"]=0
usuarios_unicos["ProbNomO%"]=0

locn=0

for usuario in usuarios_unicos["ProbNomV%"]:
    usuarios_unicos["ProbNomV%"].iloc[locn]=round((usuarios_unicos["ProbV"].iloc[locn]/usuarios_unicos["SumProb"].iloc[locn])*100,2)
    usuarios_unicos["ProbNomX%"].iloc[locn]=round((usuarios_unicos["ProbX"].iloc[locn]/usuarios_unicos["SumProb"].iloc[locn])*100,2)
    usuarios_unicos["ProbNomO%"].iloc[locn]=round((usuarios_unicos["ProbO"].iloc[locn]/usuarios_unicos["SumProb"].iloc[locn])*100,2)
    locn=locn+1

In [ ]:
#Comparar resultados obtenidos vs train

usuarios_unicos['Answer Predicted']=" "
locap=0
for user in usuarios_unicos['User']:
    xVal=usuarios_unicos['ProbNomX%'].iloc[locap]
    vVal=usuarios_unicos['ProbNomV%'].iloc[locap]
    oVal=usuarios_unicos['ProbNomO%'].iloc[locap]
    if xVal > vVal and xVal > oVal:
        usuarios_unicos['Answer Predicted'].iloc[locap]='x'
    if vVal > xVal and vVal > oVal:
        usuarios_unicos['Answer Predicted'].iloc[locap]='v'
    if oVal > vVal and oVal > xVal:
        usuarios_unicos['Answer Predicted'].iloc[locap]='o'
    locap=locap+1


In [70]:
usuarios_unicos.to_csv('Results_test.csv', index=False)

In [ ]:
df_uigrams_test.head(2)

In [ ]:
usuarios_unicos['Real Answer']=" "
locus=0
for user in usuarios_unicos["User"]:
    locni=0
    for useruni in df_uigrams_test["User"]:
        if user == useruni:
            usuarios_unicos["Real Answer"].iloc[locus]=df_uigrams_test["Recommend"].iloc[locni]
        locni=locni+1
    locus=locus+1


## Accuracy

In [ ]:
y_pred=usuarios_unicos["Answer Predicted"]
y_test=usuarios_unicos["Real Answer"]

tP=0
sUM=0
cc=0

for x in y_pred:
    sUM=sUM+1
    if x == y_test.iloc[cc]:
        tP = tP + 1
    cc=cc+1

print(f'Accuracy: {tP/sUM}')
